# 네이버 뉴스 크롤링

- ##### (주의) 이 코드에는 데이터 전처리, 크롤링한 데이터 저장이 포함되어 있지 않으며 추후 업로드 예정이다.
- 주제: BeautifulSoup를 이용하여 네이버 정치 카테고리의 뉴스를 크롤링을 하는 방법을 소개한다.
- 작성 일시: 2018-05-14
- 수정 일시: 2018-05-14
- 작성자: 부현경 (hyunkyung.boo@gmail.com)

In [1]:
from urllib.request import urlopen
import urllib
from bs4 import BeautifulSoup
import time
import json
from collections import OrderedDict
import re

In [2]:
#정치 카테고리의 네이버 뉴스 url
p_news_url = 'http://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=100'

# 정치 카테고리에 존재하는 하위 카테고리
# 청와대, 국회/정당, 북한. 행정, 국방/외교, 정치일반
_sub_catgory_politics = ['264', '265', '268', '266', '267', '269']

In [3]:
# for문을 이용하여 _seed_sub_category_url 리스트에 가 URL 주소 저장
_p_sub_category_url = []
for sub in _sub_catgory_politics:
    seed_url = p_news_url + '&sid2=' + sub
    _p_sub_category_url.append(seed_url)

In [4]:
# 저장된 링크 확인
print(_p_sub_category_url)

['http://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264', 'http://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265', 'http://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=100&sid2=268', 'http://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=100&sid2=266', 'http://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=100&sid2=267', 'http://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269']


In [5]:
# 수집할 기사의 날짜를 선택하기 위한 함수
# linkCrawling 함수에 사용된다.
today = time.strftime("%Y%m%d")
def selectDate(date):
    if date is today:
        return date
    else:
        return str(date)

In [6]:
# 링크를 통해 접속한 뉴스에 대해 크롤링을 할 기사 날짜, 제목, 내용을 가져오는 함수
# run_crawling() 함수에서 사용된다.
def newsCrawling(articleurl):
    req = urllib.request.urlopen(articleurl).read().decode('euc-kr')
    soup = BeautifulSoup(req, 'html.parser', from_encoding='utf-8')

    for article in soup.findAll("span", attrs = {"class": "t11"}):
        articleDateTime = article.find_all(text=True)
        print(articleDateTime)

    for article in soup.find_all('h3', id='articleTitle'):
        articleTitle = article.find_all(text=True)
        print(articleTitle)

    for article in soup.find_all('div', id='articleBodyContents'):
        articleContent = article.find_all(text=True)
        print(articleContent)

In [7]:
# 1페이지부터 마지막번호까지 for문을 통해 1차 카테고리 안에 존재하는 기사 링크와 해당 링크로 이동하여 뉴스를 크롤링합니다.
# 수집할 날짜를 입력해주세요.(ex. 20180408) 금일 기사를 수집할 경우 today를 입력하면 됩니다.
def run_crawling(lastnumber, date):
        for url in _p_sub_category_url:
            dateurl = url + '&date=' + selectDate(date)
            # 수집할 날짜가 포함된 링크
            print(dateurl)
            lastnumber += 1
            for pgno in range(1, lastnumber):
                try:
                    source_seed_URL = urllib.request.urlopen(dateurl + '&page=' + str(pgno))
                    soup = BeautifulSoup(source_seed_URL, 'html.parser', from_encoding="utf-8")
                    item = soup.find('div', 'list_body newsflash_body').find_all('li')
                    for k in item:
                        url = k.find('a')['href']
                        print(url)
                        #입력한 날짜의 뉴스 링크에 대해 newsCrawling(url)가 작동합니다.
                        newsCrawling(url)
                    print(pgno)
                except Exception as e:
                    print(e)

In [8]:
run_crawling(2, 20180514)

http://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&date=20180514
http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=023&aid=0003372467
['2018-05-14 11:23']
['靑 "미북정상회담 의제에 \'핵우산\' 포함"...논란일자 "모른다"']
['\n', ' 본문 내용 ', '\n', ' TV플레이어 ', '\n', ' // TV플레이어 ', '\n', '\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n', '\n\t\n\t청와대 관계자가 14일 미북정상회담 의제에 미국이 우리나라에 제공하는 핵우산이 포함됐다는 취지로 발언했다가 논란이 일자 이를 급히 취소했다.', '미국의 핵우산 공약은 ‘핵무기를 갖지 못한 동맹국이 적국으로부터 핵 공격을 받으면 자국이 보유한 핵무기로 보복 공격한다’고 선언하는 정책이다. 미국은 한미안보협의회(SCM) 공동성명 등을 통해 대한민국에 핵우산을 제공한다고 공약하고 있다.', '청와대 핵심관계자는 이날 오전 춘추관에서 기자들과 만나 ‘한반도 비핵화에는 미국의 (대 한국)핵우산이나 전략자산 전개가 포함될 수 있나’라는 물음에 대해 “그런 문제까지 포함해서 북한과 미국 사이에 이야기를 할 것”이라고 답했다.', '그는 이와 관련한 우리 정부의 입장을 묻는 말에 “우리가 한반도 비핵화에 합의한 것”이라고 답했다. 이어 취재진이 ‘핵우산’에 대한 우리 정부 입장을 거듭 확인하자 “북한과 미국 사이에 (논의할 것)”이라며 “(내가) 우리 정부 입장을 확인하지 않았다”고 즉답을 피했다.', '그러나 이 관계자는 잠시 후 청와대 출입기자들에게 문자메시지를 통해 “저는 북미회담 내용을 알지도 못하고 그럴 위치에 있지도 않다”며 “핵우산과 전략자산 전개가 북미 사이에 논의되는지 알지 

C:\Users\User\Anaconda3\lib\site-packages\bs4\__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


['2018-05-14 11:18']
['靑 "북미간 비핵화·경제지원 논의, 빠르면 빠를수록 좋다"']
['\n', ' 본문 내용 ', '\n', ' TV플레이어 ', '\n', ' // TV플레이어 ', '\n', '\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n', '\n', '"국방개혁 2.0, 대통령 보고 아닌 토론"', '© News1 방은영 디자이너', '(서울=뉴스1) 조소영 기자 = 청와대는 14일 북미간 북한의 비핵화와 그에 따른 미국의 경제지원 논의가 "빠르면 빠를수록 좋다고 생각한다"고 말했다.', '청와대 핵심관계자는 이날 춘추관에서 기자들과 만나 \'앞서 마이크 폼페이오 미국 국무장관이 언급한 북한에 대한 미국 투자 및 북한의 핵 반출 가능성이 거론되는 게 우리 정부가 예상했던 것보다 빠른 전개냐\'는 물음과 관련 "압축적으로 이뤄지는 게 (좋다)"며 이렇게 말했다.', '아울러 이 관계자는 폼페이오의 언급이 2차 세계대전 후 미국이 전개한 서유럽 부흥 지원계획인 \'마셜플랜\'과 비교되는 것을 두고 "이름이야 어떻게 붙이든 기본적으로 비핵화와 체제보장·경제의 맞교환 성격이 강하지 않느냐"며 "체제보장이라고 하는 게 단순한 안전도 있지만 더 적극적으로 이야기하면 안전을 뛰어넘어 미국을 비롯한 국제사회와 정상적 교류를 할 수 있는 걸 의미한다"고 언급했다.', '관계자는 또 한반도 비핵화는 미국의 핵우산과 주한미군의 핵전략 자산전개가 포함된 것이냐는 데에 "그런 문제까지 포함해 북미 사이에 협의를 할 것"이라며 "우리 정부 입장은 제가 확인하지 않았다"고 말했다.', '다만 이와 관련 관계자는 추후 다시 입장을 내기도 했다. 그는 "오해가 있다. 저는 핵우산·전략자산 전개가 북미 사이에 논의되는지 알지 못한다"며 "(앞선 언급은) 미국과 북한 사이에 논의할 일이라는 취지"라고 설명했다.', '국방부가 지난주 \'국방개혁 2.0\'을 문재인 대통령에게 보고했다는 것과 

['2018-05-14 10:29']
['[그래픽] 文 대통령 국정수행 지지율 76.3% (리얼미터)']
['\n', ' 본문 내용 ', '\n', ' TV플레이어 ', '\n', ' // TV플레이어 ', '\n', '\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n', '\n', '【서울=뉴시스】전진우 기자 = 리얼미터가 CBS 의뢰로 5월8일~11일 전국 19세 이상 남녀 2002명을 조사해 14일 발표한 여론조사 결과에 따르면 문재인 대통령의 지지율은 전주 주간집계 대비 1.1%p 하락한 76.3%이다. ', '618tue@newsis.com', '[사진 영상 제보받습니다]', ' 공감언론 뉴시스가 독자 여러분의 소중한 제보를 기다립니다. 뉴스 가치나 화제성이 있다고 판단되는 사진 또는 영상을 뉴시스 사진영상부(n-photo@newsis.com, 02-721-7470)로 보내주시면 적극 반영하겠습니다.', '<저작권자ⓒ 공감언론 뉴시스통신사. 무단전재-재배포 금지.>', '\n', ' // 본문 내용 ', '\n']
http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=015&aid=0003938493
['2018-05-14 10:29']
['2018-05-14 10:34']
["[청와대 오늘은] 문재인 대통령, 수보회의 주재… '적폐청산' 논의"]
['\n', ' 본문 내용 ', '\n', ' TV플레이어 ', '\n', ' // TV플레이어 ', '\n', '\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n', '\n\t\n\t[ 조미현 기자  ] ', '문재인 대통령은 14일 청와대에서 수석·보좌관 회의를 주재한다. 이 자리에서 ‘적폐 청산’ 추진 성과와 향후 추진 방향에 대해 보고와 토론이 이뤄진다.', '청와대 민정수석실은

['2018-05-14 09:40']
['폼페이오 "北 비핵화시 민간투자"에 靑 "빠를수록 좋다"']
['\n', ' 본문 내용 ', '\n', ' TV플레이어 ', '\n', ' // TV플레이어 ', '\n', '\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n', '\n\t\n\t[머니투데이 최경민  기자] [[the300]"압축적으로 이뤄지는 게 좋은 것"]', '【워싱턴=AP/뉴시스】 마이크 폼페이오 미 국무장관이 11일(현지시간) 미국 워싱턴 국무부 청사에서 열린 한미 외교장관 회의 직후 기자회견을 갖고 있다. 2018. 5. 12.   <저작권자ⓒ 공감언론 뉴시스통신사. 무단전재-재배포 금지.>', '마이크 폼페이오 미국 국무부 장관이 한반도 비핵화의 대가로 북한에 대한 \'민간투자\'를 거론한 것에 대해 "우리는 (폼페이오가 언급한 프로세스가) 빠르면 빠를 수록 좋다"고 평가했다.', '청와대 관계자는 14일 춘추관에서 기자들과 만나 "(북핵 해결이) 압축적으로 이뤄지는 게 좋은 것"이라며 이같이 말했다. 비핵화와 경제지원의 등가교환을 속도감있게 추진해야 한다는 방침의 연장선에서 나온 반응이다.', '청와대 관계자는 "(이번 북핵 협상은) 기본적으로 비핵화 문제와 체제보장, 이 문제의 맞교환 성격이 처음부터 강했다"며 "체제보장이라고 하는게 단순한 안전도 있지만, 좀 더 적극적으로 이야기 하면, 안전을 뛰어 넘어서 미국을 비롯한 국제사회와 정상적인 교류를 할 수 있는 것을 의미하는 것"이라고 설명했다.', '한편 폼페이오 장관은 13일(현지시간) 미국 폭스뉴스에 출연해 "북한이 핵무기를 완전히 폐기한다면 미국은 민간투자를 허용할 계획"이라며 "미국의 민간 부문 투자는 북한이 필요로 하는 전력수급망 구축에 큰 도움이 될 것"이라고 말했다. ', '▶', '[MT리포트] "친환경 대명사?"… 韓전기차의 딜레마', "▶'밥 잘 해주는 예쁜 아파트', 조식 프리미엄 쑥쑥", ' ',

...

['2018-05-14 11:28']
['6.13 지방선거, 한 달 앞으로']
['\n', ' 본문 내용 ', '\n', ' TV플레이어 ', '\n', ' // TV플레이어 ', '\n', '\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n', '\n', '    (서울=연합뉴스) 김현태 기자 = 6.13 전국동시지방선거를 30일 앞둔 14일 오전 서울 여의도 자유한국당 당사에 지방선거까지 남은 날짜를 표시하는 표시판이 걸려있다. 2018.5.14', '    mtkht@yna.co.kr', '▶기사제보 및 문의', '▶내 생활에 흥이나는 최신 트렌드', '\xa0\xa0', '▶사진구매 문의', '\n', ' // 본문 내용 ', '\n']
http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=001&aid=0010083746
['2018-05-14 11:27']
['6.13 지방선거, 한 달 앞으로']
['\n', ' 본문 내용 ', '\n', ' TV플레이어 ', '\n', ' // TV플레이어 ', '\n', '\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n', '\n', '    (서울=연합뉴스) 김현태 기자 = 6.13 전국동시지방선거를 30일 앞둔 14일 오전 서울 여의도 자유한국당 당사에 지방선거까지 남은 날짜를 표시하는 표시판이 걸려있다. 2018.5.14', '    mtkht@yna.co.kr', '▶기사제보 및 문의', '▶내 생활에 흥이나는 최신 트렌드', '\xa0\xa0', '▶사진구매 문의', '\n', ' // 본문 내용 ', '\n']
5
http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=421&aid=000336965

['2018-05-14 11:27']
['[경향포토] 자유한국당 로텐더홀 비상의총']
['\n', ' 본문 내용 ', '\n', ' TV플레이어 ', '\n', ' // TV플레이어 ', '\n', '\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n', '\n', '자유한국당 김성태 원내대표가 14일 국회 로텐더홀에서 열린 비상의원총회에서 드루킹 특검을 촉구하고 있다.', '<권호욱 기자 biggun@kyunghyang.com>', '▶ 경향신문 SNS ', '[트위터]', ' ', '[페이스북]', ' ▶ ', '[인기 무료만화 보기]', '©경향신문(', 'www.khan.co.kr', '), 무단전재 및 재배포 금지', '\n', ' // 본문 내용 ', '\n']
http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=025&aid=0002820491
['2018-05-14 11:27']
['2018-05-14 11:38']
["'안보이슈' 안 먹히자 경제·민생으로 달려가는 야당"]
['\n', ' 본문 내용 ', '\n', ' TV플레이어 ', '\n', ' // TV플레이어 ', '\n', '\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n', '\n\t\n\t자유한국당과 바른미래당 등 야당이 지방선거 전략을 ‘안보’에서 ‘민생’으로 선회하고 있다.    ', ' ', ' ', ' ', "서울 서초구 양재동 더케이 호텔에서 열린 '자유한국당 6.13 지방선거 서울 필승결의대회'. 무대 한복판에 '경제를 통째로 포기하시겠습니까?'라는 현수막이 자리했다. [연합뉴스]", ' ', ' ', ' ', '  남북정상회담 등으로 조성된 남북화해 국면을 기존의 ‘안보 무능’ 등의 프레임으로 돌파하기 쉽지 않다는 판단에서다. 대신 야당이 돌파구

['2018-05-14 11:26']
['[경향포토] 자유한국당 본회의장 가로 막고  비상의총']
['\n', ' 본문 내용 ', '\n', ' TV플레이어 ', '\n', ' // TV플레이어 ', '\n', '\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n', '\n', '자유한국당 김성태 원내대표가 14일 국회 로텐더홀에서 열린 비상의원총회에서 드루킹 특검을 촉구하고 있다.', '<권호욱 기자 biggun@kyunghyang.com>', '▶ 경향신문 SNS ', '[트위터]', ' ', '[페이스북]', ' ▶ ', '[인기 무료만화 보기]', '©경향신문(', 'www.khan.co.kr', '), 무단전재 및 재배포 금지', '\n', ' // 본문 내용 ', '\n']
6
http://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=269&oid=032&aid=0002869508
['2018-05-14 11:26']
['[경향포토] 자유한국당 본회의장 앞 피켓팅 농성']
['\n', ' 본문 내용 ', '\n', ' TV플레이어 ', '\n', ' // TV플레이어 ', '\n', '\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n', '\n', '자유한국당 김성태 원내대표등 소속 의원들이 14일 국회 로텐더홀에서 열린 비상의원총회에서 드루킹 특검을 촉구하고 있다.', '<권호욱 기자 biggun@kyunghyang.com>', '▶ 경향신문 SNS ', '[트위터]', ' ', '[페이스북]', ' ▶ ', '[인기 무료만화 보기]', '©경향신문(', 'www.khan.co.kr', '), 무단전재 및 재배포 금지', '\n', ' // 본문 내용 ', '\n']
http://news.naver.com/main/read.nhn?mode=LS2

['2018-05-14 11:24']
['민주당 광주시당, 전국 최초 청년비례 절차-자격 시비']
['\n', ' 본문 내용 ', '\n', ' TV플레이어 ', '\n', ' // TV플레이어 ', '\n', '\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n', '\n', '【광주=뉴시스】송창헌 기자 = 더불어민주당 광주시당이 전국 최초로 도입한 광역의원 청년비례대표제와 관련, 상상실현네트워크와 위민연구원, 지역공공정책플랫폼 광주로 등 광주지역 일부 청년단체들이 14일 광주시의회 브리핑룸에서 기자회견을 열고 "민주당 광주시당 청년비례 대표 공천 후보 자격을 박탈할 것"을 요구하고 있다. 2018.05.14 goodchang@newsis.com', '【광주=뉴시스】송창헌 기자 = 더불어민주당 광주시당이 전국 최초로 도입한 광역의원 청년비례대표제가 자격 시비에 휘말리면서 공정성 논란을 낳고 있다.', '상상실현네트워크와 위민연구원, 지역공공정책플랫폼 광주로 등 광주지역 일부 청년단체들은 14일 오전 광주시의회 브리핑룸에서 기자회견을 열고 "민주당 광주시당 청년비례 대표 공천 후보 자격을 박탈할 것"을 촉구했다.', '이들은 성명서를 통해 "민주당 광주시당이 비례대표 추천공고를 통해 참여자격을 권리당원으로 한정해 청년참여를 제한시켰고, 당초 예고했던 공개오디션도 취소해 시민평가단의 현장 참여를 제한했을 뿐 아니라 시민평가단 공개모집 대신 여론조사기관을 통한 무작위추출 방식을 택했다"며 절차상 문제를 제기했다.', '특히 "검증 부분에서도 문제점이 많다"며 "비례대표로 선출된 최영환(34)씨는 계명대 체육학과를 3학기만 다니고 중퇴했고, 필리핀 퍼시픽인터컨티넨탈칼리지(PIC)에서 학사학위를 취득했다고 신고했지만, 페이스북 프로필에는 \'전남대 경영대학원 M&A; 석사\'라고 기재됐다가 뒤늦게 삭제됐는데 이는 명백한 학력 위조"라고 주장했다.', '이어 "아시아문화원 재직시절인 2016년 10월

['2018-05-14 11:21']
['임정엽 전북지사 후보 "정부, 군산경제 회생대책 내놔야"']
['\n', ' 본문 내용 ', '\n', ' TV플레이어 ', '\n', ' // TV플레이어 ', '\n', '\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n', '\n', '임정엽 민주평화당 전북도지사 예비후보와 6·13지방선거 출마자들이 정부에 군산경제 회생을 촉구하는 기자회견을 갖고 있다.© News1', '(군산=뉴스1) 김재수 기자 = 임정엽 민주평화당 전북도지사 예비후보는 14일 정부는 군산경제 회생을 위한 특단의 대책을 마련할 것을 촉구했다.', '임 후보는 이날 군산시청 브리핑룸에서 기자회견을 갖고 "정부가 지난 10일 8000억원의 혈세를 투입하는 한국지엠의 경영정상화 방안을 최종 확정했다"면서 "하지만 정부는 협상 내내 낙후지역인 군산을 철저하게 배제하고 경남 창원과 부평공장을 살리는 방안에만 몰두했다"고 성토했다.', '이어 "군산조선소 협력업체는 82곳에서 22곳으로 줄면서 무려 64곳이 도산했고 근로자도 5250명에서 391명으로 줄어 군산경제는 초토화됐음에도 경영악화로 망하게 된 대우조선, STX조선 등은 지난 10년간 20조원이 넘는 혈세가 흘러들어갔다"고 비판했다.', '그는 또 "문재인 대통령은 후보 시절 군산조선소 폐쇄가 없을 것이라고 공약해 놓고 본인이 대통령 된 지 2개월 만에 군산조선소를 폐쇄했다"며 "\'나중에\'라는 말을 이젠 절대 믿을 수 없다"고 주장했다.', '그러면서 "군산경제 재건방안 없는 원칙 없는 지원을 평화당은 결코 용납할 수 없다"며 "평화당 군산경제비상대책위원회는 군산 경제를 재건하고 전북의 혁신성장 방안을 마련하기 위해 모든 역량을 집중하겠다"고 말했다.', 'kjs67@news1.kr', '▶ [뉴스1] 채널설정하기', ' ', '▶ 제보하기', '▶ ‘6·13지방선거’ 뉴스보기', '[© 뉴스1코리아(', 'news1.k